# Exploration des données de Sirene

## Importation des différents packages nécessaires.

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import nltk
import string
from nltk.corpus import stopwords

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importation des données

In [9]:
DBRaw = pd.read_parquet("../data/extraction_sirene_20220510.parquet", engine="pyarrow")

On transforme les valeurs manquantes en NaN.

In [7]:
DBRaw = DBRaw.fillna(value=np.nan)

On garde seulement les variables potentiellement intéressantes : 
- ``APE_SICORE :`` Code APE (Activité Principale Exercée) retenu lors du traitement de codification (soit Sicore soit gestionnaire) ;
- ``NAT_SICORE :`` Nature de l'activité de l'entreprise ;
- ``SED_SICORE :`` Sédentarité de l'entreprise ;
- ``EVT_SICORE :`` Sédentarité de l'entreprise ;
- ``LIB_SICORE :`` Sédentarité de l'entreprise ;
- ``DATE :`` Sédentarité de l'entreprise ;
- ``AUTO :`` Type de liasse extrait de la base brute Sirène ;
- ``SURF :`` Surface en $m^2$ de l'établissement.


In [8]:
Var2Keep = [
    "APE_SICORE",
    "LIB_SICORE",
    "AUTO",
    "DATE",
    "NAT_SICORE",
    "SED_SICORE",
    "EVT_SICORE",
    "SURF",
]
DB = DBRaw[Var2Keep]

On regarde l'ampleur des variables manquantes pour les deux variables principales ``APE_SICORE`` et ``LIB_SICORE``.

In [9]:
DB[["APE_SICORE", "LIB_SICORE"]].isnull().sum() / DB.shape[0] * 100

Dask Series Structure:
npartitions=1
APE_SICORE    object
LIB_SICORE       ...
dtype: object
Dask Name: mul, 9 tasks

On supprime les liasses où une valeur est manquante pour l'une de ces deux variables (6.77% de la base). Il s'agit principalement du code APE donc il n'est pas nécessaire de l'imputer.

In [14]:
DB = DB[DB["APE_SICORE"].isna()]

ValueError: The truth value of a Series is ambiguous. Use a.any() or a.all().

On a finalement 10.8 millions de liasses.

In [ ]:
DB.shape

## I- Modèle 1

On estime un modèle FastText standard en utilisant seulement les libellés comme features.

In [ ]:
DB1 = DB[["APE_SICORE", "LIB_SICORE"]]
DB1 = DB1[DB1["LIB_SICORE"].notna()]

In [ ]:
def CleanLib(lib):
    #    if not isinstance(lib,str):
    #        return "NAN"
    #    else:
    # On supprime toutes les ponctuations
    lib = lib.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # On supprime tous les chiffres
    lib = lib.translate(str.maketrans(string.digits, " " * len(string.digits)))

    # On supprime les stopwords et on renvoie les mots en majuscule
    return " ".join(
        [x.upper() for x in lib.split() if x.lower() not in stopwords.words("french") + ["a"]]
    )

In [21]:
stopwords_ = set(stopwords.words("french") + ["a"])


def CleanLib(lib):
    # On supprime toutes les ponctuations
    lib = lib.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # On supprime tous les chiffres
    lib = lib.translate(str.maketrans(string.digits, " " * len(string.digits)))

    # On supprime les stopwords et on renvoie les mots en majuscule
    return " ".join([x.upper() for x in lib.split() if x.lower() not in stopwords_])

In [28]:
DB1["LIB_CLEAN"] = DB1["LIB_SICORE"].apply(lambda x: CleanLib(x))